# Imports

In [129]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Set random seeds

In [130]:
tf.random.set_seed(42)
np.random.seed(42)

# Load data

In [131]:
raw_df = pd.read_csv("datasets/5_split/df_full.csv")

In [132]:
df = raw_df.copy()
sample_df = df.groupby('fold_id_python').sample(n = 20, random_state = 42)
sample_df = sample_df.reset_index(drop=True)

# Separate features

In [133]:
# # Dependent variables
# labels = df.pop('very_good_health')

# # CV folds
# fold_ids = df.pop("fold_id_python")
# folds = np.unique(fold_ids)

# # Independent variables
# features = df.drop(columns = ["fold_id_r"])

In [134]:
# Dependent variables
labels = sample_df.pop('very_good_health')

# CV folds
fold_ids = sample_df.pop("fold_id_python")
folds = np.unique(fold_ids)

# Independent variables
features = sample_df.drop(columns = ["fold_id_r"])

# Functions

## Get random hyperparameters

In [ ]:
def get_random_hyperparameters():
    no_of_layers = np.random.randint(1, 10)
    no_of_nodes = []
    for i in range(0, no_of_layers):
        no_of_nodes.append(np.random.randint(10, 100))
    learning_rate = np.random.uniform(0.0001, 0.01)
    epochs = np.random.randint(20, 500)
    patience = np.random.randint(5, 15)
    return no_of_layers, no_of_nodes, learning_rate, epochs, patience
    

## Build model

In [136]:
def build_model(train_features, no_of_layers, no_of_nodes, learning_rate):

    layers = []

    normaliser = keras.layers.Normalization(axis = -1)
    normaliser.adapt(np.array(train_features))
    layers.append(normaliser)

    for layer_no in range(no_of_layers):
        layers.append(keras.layers.Dense(no_of_nodes[layer_no], activation = "relu"))

    layers.append(keras.layers.Dense(1))     # Single output for regression value

    model = keras.Sequential(
        layers,
    )

    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate = learning_rate),
        loss='mse'
    )

    return model

## Build early stopper

In [137]:
def build_early_stopper(patience):
    early_stopper = keras.callbacks.EarlyStopping(
        monitor = "val_loss",
        patience = patience,
        restore_best_weights = True
    )
    return early_stopper

## Get evaluation metrics

In [138]:
def get_evaluation_metrics(val_labels, predictions):
    mae = mean_absolute_error(val_labels, predictions)
    mse = mean_squared_error(val_labels, predictions)
    r2 = r2_score(val_labels, predictions)
    return mae, mse, r2

## Get average score across cross-validation folds

In [139]:
def get_avg_scores(cv_results):
    mae_scores = []
    mse_scores = []
    r2_scores = []

    for result in cv_results:
        mae_scores.append(result["mae"])
        mse_scores.append(result["mse"])
        r2_scores.append(result["r2"])

    avg_mae = np.mean(mae_scores)
    avg_mse = np.mean(mse_scores)
    avg_r2 = np.mean(r2_scores)

    return avg_mae, avg_mse, avg_r2

## Get optimal hyperparameters

In [140]:
def get_optimal_hyperparameters(hp_combinations, cv_results):
    hp_combination_scores = []
    for i in range(len(hp_combinations)):
        current_hp_combination_results = [result for result in cv_results if result["hp_combination"] == i]
        mae, mse, r2 = get_avg_scores(current_hp_combination_results)
        hp_combination_scores.append(mse)
    optimal_combination = np.argmin(hp_combination_scores)
    optimal_hps = hp_combinations[optimal_combination]
    return optimal_hps

# Cross-validation

## Initialise HP and results arrays

In [141]:
hp_combinations = []
cv_results = []

##  Inner loop for model selection

In [142]:
for i in range(10):

    # Get hyperparameters
    no_of_layers, no_of_nodes, learning_rate, epochs, patience = get_random_hyperparameters()
    current_hps = {
        "no_of_layers": no_of_layers,
        "no_of_nodes": no_of_nodes,
        "learning_rate": learning_rate,
        "epochs": epochs,
        "patience": patience
    }
    hp_combinations.append(current_hps)

    # Inner cross-validation
    for fold in folds:
        print(f"\n --- Training model {i} on fold {fold} ---")

        # Get training and validation sets from inner fold ids
        is_in_validation_set = fold_ids == fold
        is_in_training_set = ~is_in_validation_set
        train_features = features.loc[is_in_training_set]
        train_labels = labels.loc[is_in_training_set]
        val_features = features.loc[is_in_validation_set]
        val_labels = labels.loc[is_in_validation_set]

        # Build model
        model = build_model(train_features, no_of_layers, no_of_nodes, learning_rate)
        early_stopper = build_early_stopper(patience)

        # Fit model
        model.fit(
            train_features,
            train_labels,
            epochs = epochs,
            validation_split = 0.2,     # Research whether I should use val_data for this, or whether that would lead to data leakage
            callbacks = [early_stopper],
            verbose = 0
        )

        # Get predictions using fitted model
        predictions = model.predict(val_features).flatten()

        # Get accuracy scores
        mae, mse, r2 = get_evaluation_metrics(val_labels, predictions)

        # Add scores for current fold to results
        cv_results.append({
            "hp_combination": i,
            "fold": fold,
            "hps": current_hps,
            "mae": mae,
            "mse": mse,
            "r2": r2
        })

optimal_hps = get_optimal_hyperparameters(hp_combinations, cv_results)


 --- Training model 0 on fold 0 ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step

 --- Training model 0 on fold 1 ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step

 --- Training model 0 on fold 2 ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step

 --- Training model 0 on fold 3 ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step

 --- Training model 0 on fold 4 ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step

 --- Training model 0 on fold 5 ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step

 --- Training model 0 on fold 6 ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step

 --- Training model 0 on fold 7 ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step

 --- Training model 0 on fold 8 ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step

 --- Training model 0 on fold 9 ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step

 --- Training model 1 on fold 0 ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step

 --- Training model 1 on fold 1 ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step

 --- Training model 1 on fold 2 ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step

 --- Training model 1 on

## Get optimal HPs

In [143]:
get_optimal_hyperparameters(hp_combinations, cv_results)

{'no_of_layers': 8,
 'no_of_nodes': [87, 96, 71, 49, 94, 89, 91, 62],
 'learning_rate': 0.005908836540072098,
 'epochs': 236,
 'patience': 13}

## Outer loop for model evaluation

In [144]:
# for fold in outer_folds:
#     print(f"\n --- Training on fold {fold} ---")

#     # Get training and validation sets from inner fold ids
#     is_in_validation_set = fold_ids == fold
#     is_in_training_set = ~is_in_validation_set
#     train_features = features.loc[is_in_training_set]
#     train_labels = labels.loc[is_in_training_set]
#     val_features = features.loc[is_in_validation_set]
#     val_labels = labels.loc[is_in_validation_set]

#     # Build model
#     model = build_model(train_features, no_of_layers, no_of_nodes, learning_rate)
#     early_stopper = build_early_stopper(patience)

#     # Fit model
#     model.fit(
#         train_features,
#         train_labels,
#         epochs = epochs,
#         validation_split = 0.2,     # Research whether I should use val_data for this, or whether that would lead to data leakage
#         callbacks = [early_stopper],
#         verbose = 0
#     )

#     # Get predictions using fitted model
#     predictions = model.predict(val_features).flatten()

#     # Get accuracy scores
#     mae, mse, r2 = get_evaluation_metrics(val_labels, predictions)

#     # Add scores for current fold to results
#     cv_results.append({
#         "hp_combination": i,
#         "fold": fold,
#         "hps": current_hps,
#         "mae": mae,
#         "mse": mse,
#         "r2": r2
#     })
